<a href="https://colab.research.google.com/github/ryosuke1224-haas/skip_gram_2/blob/main/Skip_grams_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import os
import string
import pandas as pd
import numpy as np
import re
import pprint
from urllib.request import urlopen

import gensim
from gensim.models import KeyedVectors
from gensim import corpora, models, similarities
import nltk
from nltk import word_tokenize, tokenize
from sklearn.manifold import TSNE
nltk.download('punkt')

!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
--2020-12-20 22:17:16--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.140.254
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.140.254|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



Generate embeddings

In [17]:
url = "http://www.gutenberg.org/cache/epub/844/pg844.txt" 
resp = urlopen(url)
raw = resp.read().decode('utf8')
firstlook = tokenize.sent_tokenize(raw)

In [18]:
pattern = r'''(?x)  # set flag to allow verbose regexps
(?:[A-Z]\.)+        # abbreviations, e.g. U.S.A.
|\w+(?:[-']\w+)*    # words with optional internal hyphens
|\$?\d+(?:\.\d+)?   # currency, e.g. $12.80 
|\.\.\.             # elipses
|[.,;"'?()-_`]      # these are separate tokens
'''
tokenized_raw = " ".join(nltk.regexp_tokenize(raw, pattern))
tokenized_raw = tokenize.sent_tokenize(tokenized_raw)

In [19]:
# Remove punctuations
nopunct = []
for sent in tokenized_raw:
    a = [w for w in sent.split() if w not in string.punctuation]
    nopunct.append(" ".join(a))

In [20]:
# Word tokenize
tok_corp = [nltk.word_tokenize(sent) for sent in nopunct]

In [21]:
unique_words = list(set([item for sublist in tok_corp for item in sublist]))

In [22]:
model = gensim.models.Word2Vec(tok_corp, min_count=1, size=16, window=5)

In [23]:
model.most_similar

<bound method BaseWordEmbeddingsModel.most_similar of <gensim.models.word2vec.Word2Vec object at 0x7fd34bfd35c0>>

In [24]:
vector_list = model[unique_words] 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [25]:
np.array(vector_list).shape

(3464, 16)

Playing with the model created

**Hypothesis 1:**
Lady Augusta Bracknell was poor in the childhood and climb up the social ladder by marring into the aristocracy. now she is very arrogant, powerful and ruthless person. So I would expect those words are cosinde similar to words such as rich, humble. Some words are not appeared in this corpus. here I compared cosine similarity of humbe and Lady Brackness, and mean and Lady Bracknell

In [26]:
print(model.similarity('Bracknell','humble'))
print(model.similarity('Bracknell','mean'))

0.7491481
0.98474985


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


**Hypothesis 2:**
 Gwendolen Fairfax is the daughter, of Lady Bracknell. She is sophisticated and confidence of a London socialite. she is very submissive to her mother. so those words can be associated with either Gwendolen or Fairfax. Here I compared confidence and greedy.

In [27]:
print(model.similarity('Gwendolen','confidence'))
print(model.similarity('Gwendolen','greedy'))

0.9670777
0.6905378


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


**Hypothesis 3:** Bunbury is a hypothetical charactor who is supposed be sickly. so he should be cosine close to weak compared to strong. 

In [28]:
print(model.similarity('Bunbury','weak'))
print(model.similarity('Bunbury','strong'))

0.8405302
0.47168854


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


Other than first hypothesis My hypothesises weren't proven to be correct. However, I think this could be partly because the corpus I chose was not big enought to represent true relationship of the word.

Use T-SNE or PCA to reduce the dimensionality of the vectors to two dimensions

In [29]:
#For Google news corpus

In [30]:
model_google = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)
word_vector=model_google.wv

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [31]:
#Use PCA
from sklearn.decomposition import PCA
df=pd.DataFrame(model_google.wv.index2word)
df_sample=df.sample(n=20000, random_state=42)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [32]:
words=[]
for i in df_sample[0]:
  words.append(i)

In [33]:
wv=[]
for i in words:
  wv.append(model_google[i])

In [34]:
pca = PCA(n_components=2)
pca.fit(wv)
feature = pca.transform(wv)

In [35]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [36]:
feature2=pd.DataFrame(nltk.pos_tag(words))
feature2.columns=['word','part of speech']
feature2.head()

,word,part of speech
0,Agilent_AFM,NNP
1,Homebuyers_Tax,NNP
2,Next_Generation_Networks_NGNs,NNP
3,Plastic_Bottles,NNP
4,Scary_Scary,NNP


In [37]:
df=pd.DataFrame(feature)
df.columns=['x','y']
df=pd.concat([df,feature2],axis=1)
df

,x,y,word,part of speech
0,-0.435964,0.237486,Agilent_AFM,NNP
1,-0.085541,0.079579,Homebuyers_Tax,NNP
2,0.034493,-0.086842,Next_Generation_Networks_NGNs,NNP
3,0.575529,0.745088,Plastic_Bottles,NNP
4,-0.202629,0.259199,Scary_Scary,NNP
...,...,...,...,...
19995,0.016821,-0.180449,bubble_gum_pop,VBP
19996,0.136985,-0.020838,Sun_NASDAQ_JAVA,NNP
19997,-0.357254,0.104705,shivs,NN
19998,-0.547161,0.136205,Outokumpu_NASDAQ_OUT1V.HE,NNP


In [38]:
from google.colab.output import eval_js
from IPython.display import Javascript
!git clone https://github.com/CAHLR/d3-scatterplot.git

Cloning into 'd3-scatterplot'...
remote: Enumerating objects: 1022, done.
remote: Total 1022 (delta 0), reused 0 (delta 0), pack-reused 1022
Receiving objects: 100% (1022/1022), 1.91 MiB | 688.00 KiB/s, done.
Resolving deltas: 100% (593/593), done.


In [39]:
cd d3-scatterplot

/content/d3-scatterplot


In [40]:
ls

dist/        index.html   package.json       README.md  vendor/
example.tsv  math.min.js  package-lock.json  src/       webpack.config.js


In [41]:
df.to_csv('mytext.tsv', sep='\t', index=False)

In [42]:
df

,x,y,word,part of speech
0,-0.435964,0.237486,Agilent_AFM,NNP
1,-0.085541,0.079579,Homebuyers_Tax,NNP
2,0.034493,-0.086842,Next_Generation_Networks_NGNs,NNP
3,0.575529,0.745088,Plastic_Bottles,NNP
4,-0.202629,0.259199,Scary_Scary,NNP
...,...,...,...,...
19995,0.016821,-0.180449,bubble_gum_pop,VBP
19996,0.136985,-0.020838,Sun_NASDAQ_JAVA,NNP
19997,-0.357254,0.104705,shivs,NN
19998,-0.547161,0.136205,Outokumpu_NASDAQ_OUT1V.HE,NNP


In [43]:
ls

dist/        math.min.js   package-lock.json  vendor/
example.tsv  mytext.tsv    README.md          webpack.config.js
index.html   package.json  src/


In [44]:
def show_port(port, data_file, width=1000, height=1200):
  display(Javascript("""
  (async ()=>{
    fm = document.createElement('iframe')
    fm.src = await google.colab.kernel.proxyPort(%d) + '/index.html?dataset=%s'
    fm.width = '90%%'
    fm.height = '%d'
    fm.frameBorder = 0
    document.body.append(fm)
  })();
  """ % (port, data_file, height)))

port = 8000
data_file = 'mytext.tsv'
height = 1600

get_ipython().system_raw('cd d3-scatterplot && python3 -m http.server %d &' % port) 
show_port(port, data_file, height)

<IPython.core.display.Javascript object>

In [45]:
data_embed_google=TSNE(n_components=2, perplexity=50, verbose=2, method='barnes_hut').fit_transform(wv)
data_embed_google
df_google=pd.DataFrame(data_embed_google)
df_google.columns=['x','y']

[t-SNE] Computing 151 nearest neighbors...
[t-SNE] Indexed 20000 samples in 1.188s...
[t-SNE] Computed neighbors for 20000 samples in 326.547s...
[t-SNE] Computed conditional probabilities for sample 1000 / 20000
[t-SNE] Computed conditional probabilities for sample 2000 / 20000
[t-SNE] Computed conditional probabilities for sample 3000 / 20000
[t-SNE] Computed conditional probabilities for sample 4000 / 20000
[t-SNE] Computed conditional probabilities for sample 5000 / 20000
[t-SNE] Computed conditional probabilities for sample 6000 / 20000
[t-SNE] Computed conditional probabilities for sample 7000 / 20000
[t-SNE] Computed conditional probabilities for sample 8000 / 20000
[t-SNE] Computed conditional probabilities for sample 9000 / 20000
[t-SNE] Computed conditional probabilities for sample 10000 / 20000
[t-SNE] Computed conditional probabilities for sample 11000 / 20000
[t-SNE] Computed conditional probabilities for sample 12000 / 20000
[t-SNE] Computed conditional probabilities for 

In [46]:
df_google=pd.concat([df_google,feature2],axis=1)
df

,x,y,word,part of speech
0,-0.435964,0.237486,Agilent_AFM,NNP
1,-0.085541,0.079579,Homebuyers_Tax,NNP
2,0.034493,-0.086842,Next_Generation_Networks_NGNs,NNP
3,0.575529,0.745088,Plastic_Bottles,NNP
4,-0.202629,0.259199,Scary_Scary,NNP
...,...,...,...,...
19995,0.016821,-0.180449,bubble_gum_pop,VBP
19996,0.136985,-0.020838,Sun_NASDAQ_JAVA,NNP
19997,-0.357254,0.104705,shivs,NN
19998,-0.547161,0.136205,Outokumpu_NASDAQ_OUT1V.HE,NNP


In [47]:
df_google.to_csv('mytext3.tsv', sep='\t', index=False)

In [48]:
df_google

,x,y,word,part of speech
0,-1.240660,0.600790,Agilent_AFM,NNP
1,3.019239,-5.529842,Homebuyers_Tax,NNP
2,-11.457996,-27.678257,Next_Generation_Networks_NGNs,NNP
3,16.135761,-7.101675,Plastic_Bottles,NNP
4,1.575932,13.762531,Scary_Scary,NNP
...,...,...,...,...
19995,10.627417,15.976443,bubble_gum_pop,VBP
19996,-7.954498,-40.964947,Sun_NASDAQ_JAVA,NNP
19997,-23.792864,17.578207,shivs,NN
19998,-3.464839,4.614738,Outokumpu_NASDAQ_OUT1V.HE,NNP


In [49]:
port = 8000
data_file = 'mytext3.tsv'
height = 1600

get_ipython().system_raw('cd d3-scatterplot && python3 -m http.server %d &' % port) 
show_port(port, data_file, height)

<IPython.core.display.Javascript object>

In [50]:
# Lets dim reduce the 16 dimension vectors to 2 dimensions to vizualise the dataset 
data_embed=TSNE(n_components=2, perplexity=50, verbose=2, method='barnes_hut').fit_transform(vector_list)

[t-SNE] Computing 151 nearest neighbors...
[t-SNE] Indexed 3464 samples in 0.011s...
[t-SNE] Computed neighbors for 3464 samples in 0.514s...
[t-SNE] Computed conditional probabilities for sample 1000 / 3464
[t-SNE] Computed conditional probabilities for sample 2000 / 3464
[t-SNE] Computed conditional probabilities for sample 3000 / 3464
[t-SNE] Computed conditional probabilities for sample 3464 / 3464
[t-SNE] Mean sigma: 0.030142
[t-SNE] Computed conditional probabilities in 0.561s
[t-SNE] Iteration 50: error = 77.4128418, gradient norm = 0.0287649 (50 iterations in 2.471s)
[t-SNE] Iteration 100: error = 73.3984375, gradient norm = 0.0088746 (50 iterations in 1.506s)
[t-SNE] Iteration 150: error = 73.1803741, gradient norm = 0.0062986 (50 iterations in 1.243s)
[t-SNE] Iteration 200: error = 73.0941620, gradient norm = 0.0042932 (50 iterations in 1.204s)
[t-SNE] Iteration 250: error = 73.0389404, gradient norm = 0.0015361 (50 iterations in 1.151s)
[t-SNE] KL divergence after 250 iterat

In [51]:
data_embed
df_text=pd.DataFrame(data_embed)
df_text.columns=['x','y']
feature3=pd.DataFrame(nltk.pos_tag(unique_words))
feature3.columns=['word','part of speech']
feature3

,word,part of speech
0,Bunbury,NNP
1,found,VBD
2,scissors,NNS
3,acres,NNS
4,foolish,JJ
...,...,...
3459,politics,NNS
3460,That,WDT
3461,stress,VBP
3462,fortunate,JJ


In [52]:
df_text=pd.concat([df_text,feature3],axis=1)
df_text

,x,y,word,part of speech
0,-49.558891,-3.463325,Bunbury,NNP
1,-30.034786,-39.363461,found,VBD
2,9.324636,27.067638,scissors,NNS
3,4.127422,29.468616,acres,NNS
4,11.990938,-14.271642,foolish,JJ
...,...,...,...,...
3459,4.530085,43.460308,politics,NNS
3460,-49.228931,-5.147581,That,WDT
3461,11.705072,3.445977,stress,VBP
3462,10.062011,21.642498,fortunate,JJ


In [53]:
ls

dist/        math.min.js  package.json       src/
example.tsv  mytext3.tsv  package-lock.json  vendor/
index.html   mytext.tsv   README.md          webpack.config.js


In [54]:
df_text.to_csv('mytext2.tsv', sep='\t', index=False)

In [55]:
port = 8000
data_file = 'mytext2.tsv'
height = 1600

get_ipython().system_raw('cd d3-scatterplot && python3 -m http.server %d &' % port) 
show_port(port, data_file, height)

<IPython.core.display.Javascript object>